# simple nn model fitting

Notebook will run a very simple nn on the training points

In [1]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# import librosa
# import librosa.display
# import sklearn as skl
# import sklearn.utils
# import IPython.display as ipd
# from sklearn.model_selection import train_test_split
# import ast

from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#define network class
#used some help from https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

class Net(nn.Module):
    def __init__(self, input_size, output_size = 1):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_size, int(input_size/2))
        self.layer2 = nn.Linear(int(input_size/2), int(input_size/4))
        self.layer3 = nn.Linear(int(input_size/4), output_size)

    # x represents our data
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

        

In [11]:
#import data
train_data = torch.tensor(np.load('data/train_data.npy').T).float()[:1000]
labels = torch.tensor(np.load('data/train_labels.npy')).float()
n_dim, n_training_points = train_data.shape
print("n_dim = ", n_dim)
print('n_training_points = ', n_training_points )

#need to one-hot-encode labels because the numbers are labels not numerical
#got help from https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
# labels = F.one_hot(labels)
# print(labels.shape)

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(labels.unsqueeze(1)) #takes a 2d array as input and labels is 1d
enc.categories_
labels = enc.transform(labels.unsqueeze(1)).toarray()
labels = torch.tensor(labels).float()
print(labels.shape)
_, n_categories = labels.shape #number of categories is out output size for our network

n_dim =  1000
n_training_points =  50
torch.Size([50, 13])


In [12]:
model = Net(n_dim, n_categories)
print(model)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()

Net(
  (layer1): Linear(in_features=1000, out_features=500, bias=True)
  (layer2): Linear(in_features=500, out_features=250, bias=True)
  (layer3): Linear(in_features=250, out_features=13, bias=True)
)


In [15]:
max_epochs = 20
for epoch in range(max_epochs):
    optimizer.zero_grad()   # zero the gradient buffers
#     print(train_data.shape)
    output = model(train_data.T)
#     print(output.shape)
#     print(labels.shape)
#     print(type(output))
#     print(type(labels))
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()    # Does the update
    print("loss = ", loss)

loss =  tensor(0.0250, grad_fn=<MseLossBackward>)
loss =  tensor(0.0247, grad_fn=<MseLossBackward>)
loss =  tensor(0.0244, grad_fn=<MseLossBackward>)
loss =  tensor(0.0241, grad_fn=<MseLossBackward>)
loss =  tensor(0.0238, grad_fn=<MseLossBackward>)
loss =  tensor(0.0235, grad_fn=<MseLossBackward>)
loss =  tensor(0.0232, grad_fn=<MseLossBackward>)
loss =  tensor(0.0229, grad_fn=<MseLossBackward>)
loss =  tensor(0.0227, grad_fn=<MseLossBackward>)
loss =  tensor(0.0224, grad_fn=<MseLossBackward>)
loss =  tensor(0.0222, grad_fn=<MseLossBackward>)
loss =  tensor(0.0221, grad_fn=<MseLossBackward>)
loss =  tensor(0.0219, grad_fn=<MseLossBackward>)
loss =  tensor(0.0214, grad_fn=<MseLossBackward>)
loss =  tensor(0.0213, grad_fn=<MseLossBackward>)
loss =  tensor(0.0211, grad_fn=<MseLossBackward>)
loss =  tensor(0.0207, grad_fn=<MseLossBackward>)
loss =  tensor(0.0206, grad_fn=<MseLossBackward>)
loss =  tensor(0.0203, grad_fn=<MseLossBackward>)
loss =  tensor(0.0201, grad_fn=<MseLossBackward>)


In [ ]:
print(labels)